# Midterm Project - Saketh Lakshmanan

## First we write code for the BRUTE FORCE ALGORITHM

### Load and collect all data

In [1]:
import csv
import sys
import numpy as np
from itertools import chain, combinations
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
import time

def load_csv(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        return list(reader)

def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [2]:
list_input = input("Enter a dataset to analyze: Amazon, Best Buy, Kmart, Nike, or General: ")
if list_input.lower() == 'amazon':
    itemset_list = pd.read_csv('amazonlist.csv')
    the_transactions = pd.read_csv('amazontransactions.csv')
elif list_input.lower() == 'best buy':
    itemset_list = pd.read_csv('bestbuylist.csv')
    the_transactions = pd.read_csv('bestbuytransactions.csv')
elif list_input.lower() in ['kmart', 'k-mart']:
    itemset_list = pd.read_csv('kmartlist.csv')
    the_transactions = pd.read_csv('kmarttransactions.csv')
elif list_input.lower() == 'nike':
    itemset_list = pd.read_csv('nikelist.csv')
    the_transactions = pd.read_csv('niketransactions.csv')
elif list_input.lower() == 'general':
    itemset_list = pd.read_csv('generallist.csv')
    the_transactions = pd.read_csv('generaltransactions.csv')
else:
    print("Enter Valid Info (correct spelling)")
    sys.exit()

order = sorted(itemset_list['ItemName'])

dataset = []
for lines in the_transactions['Transaction']:
    trans = list(lines.strip().split(','))
    trans1 = list(np.unique(trans))
    trans1.sort(key=lambda x: order.index(x.strip()))
    dataset.append(sorted(trans1))

transaction_num = len(dataset)

min_support = int(input("Enter the minimum support count (integer, percent between 1 and 100): "))
min_confidence = float(input("Enter the minimum confidence (decimal, percent between 0 and 100): "))

Enter a dataset to analyze: Amazon, Best Buy, Kmart, Nike, or General:  kmart
Enter the minimum support count (integer, percent between 1 and 100):  40
Enter the minimum confidence (decimal, percent between 0 and 100):  45


### Initialize the dictionaries

In [3]:
candidate_itemsets = {}               
freq_itemsets = {}    
support_count_L = {}   
itemset_size = 1     # set the initial itemset size
non_frequent = {itemset_size: []}

# populate candidate itemsets with singleton itemsets
candidate_itemsets[itemset_size] = [[f] for f in order]

# print the initialized candidate dictionary for verification
print("Candidate Itemsets:", candidate_itemsets)

Candidate Itemsets: {1: [['Bed Skirts'], ['Bedding Collections'], ['Bedspreads'], ['Decorative Pillows'], ['Embroidered Bedspread'], ['Kids Bedding'], ['Quilts'], ['Shams'], ['Sheets'], ['Towels']]}


### Define a function for finding frequent itemsets

In [4]:
def frequent_itemsets(candidate_itemsets, transactions, minimum_support, non_frequent):
    # Initialize lists to store frequent itemsets, their support counts, and new non-frequent itemsets
    frequent_itemsets_list = []
    support_count_list = []
    new_non_frequent = []
    
    # Get the total number of transactions in the dataset
    transaction_num = len(transactions)
    # Get the current itemset size
    K = len(non_frequent.keys())
    
    # Iterate through each candidate itemset
    for i in range(0, len(candidate_itemsets)):
        # Flag to check if the candidate itemset is a subset of any non-frequent itemset
        is_candidate_set = 0
        
        # Check against non-frequent itemsets of the previous size (if any)
        if K > 0:
            for j in non_frequent[K]:
                if set(j).issubset(set(candidate_itemsets[i])):
                    is_candidate_set = 1
                    break
        
        # If the candidate itemset is not a subset of any non-frequent itemset, proceed
        if is_candidate_set == 0:
            # Count the occurrences of the candidate itemset in the transactions
            frequent_count = count_items(candidate_itemsets[i], transactions)
            
            # Check if the frequent count meets the minimum support threshold
            if frequent_count >= (minimum_support / 100) * transaction_num:
                # Append the frequent itemset and its support count to the lists
                frequent_itemsets_list.append(candidate_itemsets[i])
                support_count_list.append(frequent_count)
            else:
                # If not frequent, add the itemset to the list of new non-frequent itemsets
                new_non_frequent.append(candidate_itemsets[i])
    
    # Return the lists containing frequent itemsets, their support counts, and new non-frequent itemsets
    return frequent_itemsets_list, support_count_list, new_non_frequent

### Define a function for counting items and a table

In [5]:
def count_items(candidate_itemset, transactions):
    count = 0
    for i in range (0, len(transactions)):
        if set(candidate_itemset).issubset(set(transactions[i])):
            count += 1
    return count

In [6]:
def print_table (table, support_count):
    print("Itemset | Count")
    for i in range (0, len(table)):
        print("{} : {}".format(table[i], support_count[i]))
    print("\n\n")

### Define a function to link together itemsets

In [7]:
def join_itemsets(itemset1, itemset2, order):
    itemset1.sort(key=lambda x: order.index(x))
    itemset2.sort(key=lambda x: order.index(x))
    
    for i in range(len(itemset1) - 1):
        if itemset1[i] != itemset2[i]:
            return []
    
    if order.index(itemset1[-1]) < order.index(itemset2[-1]):
        return itemset1 + [itemset2[-1]]
    
    return []

### Define a function to find the candidate itemset

In [8]:
def get_candidate_set(frequent_itemsets, order):
    candidate_set = []
    for i in range(len(frequent_itemsets)):
        for j in range(i + 1, len(frequent_itemsets)):
            joined_itemset = join_itemsets(frequent_itemsets[i], frequent_itemsets[j], order)
            
            if len(joined_itemset) > 0:
                candidate_set.append(joined_itemset)
    
    return candidate_set

### Define a function to find all possible subsets of a set

In [9]:
from itertools import combinations, chain

def possible_subsets(s):
    a = list(s)
    subsets = list(chain.from_iterable(combinations(a, r) for r in range(1, len(a) + 1)))
    return subsets

### Rule formatting to look nicer

In [10]:
def write_association_rules(lhs_itemset, rhs_itemset, confidence, support, transaction_num, rule_number):
    association_rules_str = (
        f"Rule {rule_number}: {list(lhs_itemset)} -> {list(rhs_itemset)}\n"
        f"Confidence: {confidence * 100:.2f}%\n"
        f"Support: {(support / transaction_num) * 100:.2f}%\n\n"
    )
    return association_rules_str

### Find the frequent itemsets and the corresponding support then update

In [11]:
frequent_set, support, new_non_frequent = frequent_itemsets(candidate_itemsets[itemset_size], dataset, min_support, non_frequent)

freq_itemsets[itemset_size] = frequent_set
non_frequent[itemset_size] = new_non_frequent
support_count_L[itemset_size] = support           

### Define and print a table for itemsets

In [12]:
def print_itemset_table(itemset_type, itemsets, support_counts):
    print(f"\nTable {itemset_type}:\n")
    for i, itemset in enumerate(itemsets):
        itemset_str = ','.join(itemset)
        count = support_counts[i]
        print(f"{itemset_str} : {count}")

In [13]:
print_itemset_table("Candidate", candidate_itemsets[1], [count_items(item, dataset) for item in candidate_itemsets[1]])
print_itemset_table("Frequent", freq_itemsets[1], support_count_L[1])


Table Candidate:

Bed Skirts : 11
Bedding Collections : 7
Bedspreads : 7
Decorative Pillows : 10
Embroidered Bedspread : 6
Kids Bedding : 12
Quilts : 8
Shams : 10
Sheets : 10
Towels : 0

Table Frequent:

Bed Skirts : 11
Decorative Pillows : 10
Kids Bedding : 12
Quilts : 8
Shams : 10
Sheets : 10


### Generate candidate itemsets iteratively using the BRUTE FORCE  ALGORITHM

In [14]:
start_time = time.time() ## Starting the time calc here
# Initialize variables for the brute-force approach
K = itemset_size + 1  # Increment the itemset size
candidate_set = 0  # Flag to check if there are frequent itemsets
candidate_itemsets = []  # List to store candidate itemsets at each iteration

# Continue generating candidate itemsets until none are frequent for the current itemset size
while candidate_set == 0:
    # Generate candidate itemsets using the brute-force method
    candid_itemsets = get_candidate_set(freq_itemsets[K-1], order)
    # Append the generated candidate itemsets to the list
    candidate_itemsets.append(candid_itemsets)
    print("Table Candidate Itemsets {}: \n".format(K))
    # Print the table of candidate itemsets along with their support counts
    print_table(candid_itemsets, [count_items(item, dataset) for item in candid_itemsets])
    
    # Calculate frequent itemsets and update dictionaries
    frequent_set, support, new_non_frequent = frequent_itemsets(candidate_itemsets[-1], dataset, min_support, non_frequent)
    # Update dictionaries with the frequent itemsets, non-frequent itemsets, and support counts
    freq_itemsets.update({K: frequent_set})
    non_frequent.update({K: new_non_frequent})
    support_count_L.update({K: support})
    
    # Check if there are no frequent itemsets for the current itemset size
    if len(freq_itemsets[K]) == 0:
        candidate_set = 1  # Set the flag to terminate the loop
    else:
        print(f"Table Frequent Itemsets {K}:\n")
        # Print the table of frequent itemsets along with their support counts
        print_table(freq_itemsets[K], support_count_L[K])
    
    K += 1  # Increment the itemset size for the next iteration

Table Candidate Itemsets 2: 

Itemset | Count
['Bed Skirts', 'Decorative Pillows'] : 4
['Bed Skirts', 'Kids Bedding'] : 10
['Bed Skirts', 'Quilts'] : 0
['Bed Skirts', 'Shams'] : 8
['Bed Skirts', 'Sheets'] : 9
['Decorative Pillows', 'Kids Bedding'] : 4
['Decorative Pillows', 'Quilts'] : 6
['Decorative Pillows', 'Shams'] : 4
['Decorative Pillows', 'Sheets'] : 2
['Kids Bedding', 'Quilts'] : 2
['Kids Bedding', 'Shams'] : 8
['Kids Bedding', 'Sheets'] : 10
['Quilts', 'Shams'] : 1
['Quilts', 'Sheets'] : 1
['Shams', 'Sheets'] : 6



Table Frequent Itemsets 2:

Itemset | Count
['Bed Skirts', 'Kids Bedding'] : 10
['Bed Skirts', 'Shams'] : 8
['Bed Skirts', 'Sheets'] : 9
['Kids Bedding', 'Shams'] : 8
['Kids Bedding', 'Sheets'] : 10



Table Candidate Itemsets 3: 

Itemset | Count
['Bed Skirts', 'Kids Bedding', 'Shams'] : 7
['Bed Skirts', 'Kids Bedding', 'Sheets'] : 9
['Bed Skirts', 'Shams', 'Sheets'] : 6
['Kids Bedding', 'Shams', 'Sheets'] : 6



Table Frequent Itemsets 3:

Itemset | Count
['Bed S

### Generate the final association rules for BRUTE FORCE ALGORITHM

In [15]:
# Initialize variables for final association rules
final_association_rules = ""
rule_number = 1

# Loop over each itemset size (starting from 1) in the frequent itemsets
for itemset_size in range(1, len(freq_itemsets)):
    # Loop over each itemset in the current itemset size
    for itemset in freq_itemsets[itemset_size]:
        # Generate all possible non-empty subsets of the current itemset
        subsets = list(possible_subsets(set(itemset)))
        subsets.pop()  # Remove the empty set

        # Loop over each subset of the current itemset
        for subset in subsets:
            # Create item1 as a set representing the current subset
            item1 = set(subset)
            # Convert the current itemset to a set
            itemset_set = set(itemset)
            # Generate item2 as the complement of item1 in the itemset
            item2 = itemset_set - item1
            
            # Calculate support counts for the itemsets and subsets
            support_frequent_set = count_items(itemset_set, dataset)
            support_item1 = count_items(item1, dataset)
            support_item2 = count_items(item2, dataset)

            # Calculate confidence of the association rule
            confidence = support_frequent_set / support_item1

            # Check if the confidence and support meet the user-defined thresholds
            if confidence >= (min_confidence / 100) and support_frequent_set >= (min_support / 100) * transaction_num:
                # Generate and append the association rule to the final result
                final_association_rules += write_association_rules(item2, item1, confidence, support_frequent_set, transaction_num, rule_number)
                # Increment the rule number for the next association rule
                rule_number += 1

end_time = time.time() ## Ending the time calc here
elapsed_time = end_time - start_time

# Print the final association rules
print("Final Association Rules: \n")
print(final_association_rules)
print(f"\nProgram execution time: {elapsed_time:.4f} seconds")

Final Association Rules: 

Rule 1: ['Bed Skirts'] -> ['Kids Bedding']
Confidence: 83.33%
Support: 50.00%

Rule 2: ['Kids Bedding'] -> ['Bed Skirts']
Confidence: 90.91%
Support: 50.00%

Rule 3: ['Shams'] -> ['Bed Skirts']
Confidence: 72.73%
Support: 40.00%

Rule 4: ['Bed Skirts'] -> ['Shams']
Confidence: 80.00%
Support: 40.00%

Rule 5: ['Sheets'] -> ['Bed Skirts']
Confidence: 81.82%
Support: 45.00%

Rule 6: ['Bed Skirts'] -> ['Sheets']
Confidence: 90.00%
Support: 45.00%

Rule 7: ['Shams'] -> ['Kids Bedding']
Confidence: 66.67%
Support: 40.00%

Rule 8: ['Kids Bedding'] -> ['Shams']
Confidence: 80.00%
Support: 40.00%

Rule 9: ['Sheets'] -> ['Kids Bedding']
Confidence: 83.33%
Support: 50.00%

Rule 10: ['Kids Bedding'] -> ['Sheets']
Confidence: 100.00%
Support: 50.00%

Rule 11: ['Bed Skirts', 'Sheets'] -> ['Kids Bedding']
Confidence: 75.00%
Support: 45.00%

Rule 12: ['Kids Bedding', 'Sheets'] -> ['Bed Skirts']
Confidence: 81.82%
Support: 45.00%

Rule 13: ['Kids Bedding', 'Bed Skirts'] -> ['

## Verify the results with APRIORI built-in package

In [16]:
from mlxtend.frequent_patterns import apriori as mlxtend_apriori
from mlxtend.frequent_patterns import association_rules as mlxtend_association_rules
from mlxtend.preprocessing import TransactionEncoder

start_time = time.time() ## Starting the time calc here

# Use Apriori implementation from mlxtend
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
mlxtend_freq_itemsets = mlxtend_apriori(df, min_support=(min_support / 100), use_colnames=True)

# Extract association rules from frequent itemsets using mlxtend
mlxtend_rules = mlxtend_association_rules(mlxtend_freq_itemsets, metric="confidence", min_threshold=min_confidence / 100)

# Initialize variables for final association rules (mlxtend)
final_association_rules_mlxtend = ""
rule_number_mlxtend = 1

# Loop over each rule generated by mlxtend
for idx, row in mlxtend_rules.iterrows():
    # Extract items and details from mlxtend result
    item1_mlxtend = set(row['antecedents'])
    item2_mlxtend = set(row['consequents'])
    confidence_mlxtend = row['confidence']
    support_mlxtend = row['support'] * transaction_num  # Convert support to count

    # Check if the confidence and support meet the user-defined thresholds
    if confidence_mlxtend >= (min_confidence / 100) and support_mlxtend >= (min_support / 100) * transaction_num:
        # Generate and append the association rule to the final result (mlxtend)
        final_association_rules_mlxtend += write_association_rules(item2_mlxtend, item1_mlxtend, confidence_mlxtend, support_mlxtend, transaction_num, rule_number_mlxtend)
        # Increment the rule number for the next association rule
        rule_number_mlxtend += 1

end_time = time.time() ## Ending the time calc here
elapsed_time = end_time - start_time

# Print the final association rules from mlxtend
print("Final Association Rules (Apriori): \n")
print(final_association_rules_mlxtend)
print(f"\nProgram execution time: {elapsed_time:.4f} seconds")


Final Association Rules (Apriori): 

Rule 1: ['Bed Skirts'] -> ['Kids Bedding']
Confidence: 83.33%
Support: 50.00%

Rule 2: ['Kids Bedding'] -> ['Bed Skirts']
Confidence: 90.91%
Support: 50.00%

Rule 3: ['Shams'] -> ['Bed Skirts']
Confidence: 72.73%
Support: 40.00%

Rule 4: ['Bed Skirts'] -> ['Shams']
Confidence: 80.00%
Support: 40.00%

Rule 5: ['Sheets'] -> ['Bed Skirts']
Confidence: 81.82%
Support: 45.00%

Rule 6: ['Bed Skirts'] -> ['Sheets']
Confidence: 90.00%
Support: 45.00%

Rule 7: ['Shams'] -> ['Kids Bedding']
Confidence: 66.67%
Support: 40.00%

Rule 8: ['Kids Bedding'] -> ['Shams']
Confidence: 80.00%
Support: 40.00%

Rule 9: ['Sheets'] -> ['Kids Bedding']
Confidence: 83.33%
Support: 50.00%

Rule 10: ['Kids Bedding'] -> ['Sheets']
Confidence: 100.00%
Support: 50.00%

Rule 11: ['Sheets'] -> ['Kids Bedding', 'Bed Skirts']
Confidence: 90.00%
Support: 45.00%

Rule 12: ['Bed Skirts'] -> ['Kids Bedding', 'Sheets']
Confidence: 90.00%
Support: 45.00%

Rule 13: ['Kids Bedding'] -> ['Bed 

## Verify Results using FP GROWTH built-in package

In [17]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

start_time = time.time() ## Starting the time calc here

# Use TransactionEncoder for one-hot encoding
te = TransactionEncoder()
one_hot_df = te.fit_transform(dataset)
one_hot_df = pd.DataFrame(one_hot_df, columns=te.columns_)

# Find frequent itemsets using FPGrowth
frequent_itemsets_mlxtend = fpgrowth(one_hot_df, min_support=min_support / 100, use_colnames=True)
print(f"\nFrequent Itemsets using FPGrowth:")
print(frequent_itemsets_mlxtend)

# Generate association rules
rules_mlxtend = association_rules(frequent_itemsets_mlxtend, metric="confidence", min_threshold=min_confidence / 100)

end_time = time.time() ## Ending the time calc here
elapsed_time = end_time - start_time

print("\nAssociation Rules using FPGrowth:")
print(rules_mlxtend)
print(f"\nProgram execution time: {elapsed_time:.4f} seconds") 


Frequent Itemsets using FPGrowth:
    support                            itemsets
0      0.50                (Decorative Pillows)
1      0.40                            (Quilts)
2      0.60                      (Kids Bedding)
3      0.55                        (Bed Skirts)
4      0.50                            (Sheets)
5      0.50                             (Shams)
6      0.50          (Kids Bedding, Bed Skirts)
7      0.50              (Kids Bedding, Sheets)
8      0.45                (Bed Skirts, Sheets)
9      0.45  (Kids Bedding, Bed Skirts, Sheets)
10     0.40                 (Bed Skirts, Shams)
11     0.40               (Kids Bedding, Shams)

Association Rules using FPGrowth:
                   antecedents                 consequents  \
0               (Kids Bedding)                (Bed Skirts)   
1                 (Bed Skirts)              (Kids Bedding)   
2               (Kids Bedding)                    (Sheets)   
3                     (Sheets)              (Kids Bedding)